In [1]:
#Import necessary packages.

import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM
from keras.models import load_model, Sequential
from keras.optimizers import Adam
import os
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

current_directory = os.getcwd()

/Accounts/huanga2/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#Load datasets.
os.chdir(current_directory)
X_train = np.load("Training_data.npy")
X_test = np.load("Test_data.npy")
Y_train = np.load('Training_labels.npy')
Y_test = np.load('Test_labels.npy')
human_train = np.load('Human_Test.npy')
human_test = np.load('Human_Test_Labels.npy')
human_data_reference = np.load('human_Data_Reference.npy')

human_test_dict = {}

for i in range(len(human_data_reference)):
    if human_data_reference[i][0] not in human_test_dict:
        human_test_dict[human_data_reference[i][0]] = [[], []]
    human_test_dict[human_data_reference[i][0]][0].append(human_train[i])
    human_test_dict[human_data_reference[i][0]][1].append(human_test[i])
for item in human_test_dict:
    human_test_dict[item][0] = np.array(human_test_dict[item][0])
    human_test_dict[item][1] = np.array(human_test_dict[item][1])
    
print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))



(470250, 512)
(52250, 512)
470250
52250


In [5]:
#Regularize the data.

mean = X_train.mean(axis = 0)
X_train -= mean
X_test -= mean
for item in human_test_dict:
    human_test_dict[item][0] -= mean 

std = X_train.std(axis = 0)
X_train /= std
X_test /= std
for item in human_test_dict:
    human_test_dict[item][0] /= std

In [125]:
#[OPTIONAL] Choose a smaller sample.

train_data_selection = int(X_train.shape[0]*0.1)
test_data_selection = int(X_test.shape[0]*0.1)

X_train = X_train[:train_data_selection]
Y_train = Y_train[:train_data_selection]
X_test = X_test[:test_data_selection]
Y_test = Y_test[:test_data_selection]

print(X_train.shape)
print(X_test.shape)
print(len(Y_train))
print(len(Y_test))

(47027, 512)
(5225, 512)
47027
5225


In [6]:
#Train 512 x 512 network models.
        
for l1 in range(9, 10):
    model = Sequential()
    model.add(Dense(2 ** l1, activation = 'relu', input_shape = (512,)))
    model.add(Dropout(0.5))
    model.add(Dense(2 ** l1, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = Adam(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
        
    history = model.fit(X_train, Y_train, epochs = 1500, batch_size = 4096, shuffle = True,
                                validation_split = 0.1, verbose = 2)
        
    os.chdir('/data')
    model.save('{}x{}_model.h5'.format(2 ** l1, 2 ** l1))
        
    os.chdir('/data')
    with open('{}x{}_model_history.pickle'.format(2 ** l1, 2 ** l1), 'wb') as dictionary:
        pickle.dump(history, dictionary, protocol=pickle.HIGHEST_PROTOCOL)
            

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 513       
Total params: 525,825
Trainable params: 525,825
Non-trainable params: 0
_________________________________________________________________
Train on 423225 samples, validate on 47025 samples
Epoch 1/1500
 - 8s - loss: 0.7052 - acc: 0.6050 - val_loss: 0.6099 - val_acc: 0.6671


Epoch 84/1500
 - 14s - loss: 0.5371 - acc: 0.7169 - val_loss: 0.5637 - val_acc: 0.7150
Epoch 85/1500
 - 14s - loss: 0.5370 - acc: 0.7174 - val_loss: 0.5642 - val_acc: 0.7150
Epoch 86/1500
 - 14s - loss: 0.5365 - acc: 0.7178 - val_loss: 0.5639 - val_acc: 0.7146
Epoch 87/1500
 - 14s - loss: 0.5362 - acc: 0.7181 - val_loss: 0.5626 - val_acc: 0.7143
Epoch 88/1500
 - 13s - loss: 0.5351 - acc: 0.7185 - val_loss: 0.5633 - val_acc: 0.7147
Epoch 89/1500
 - 14s - loss: 0.5352 - acc: 0.7188 - val_loss: 0.5631 - val_acc: 0.7148
Epoch 90/1500
 - 15s - loss: 0.5338 - acc: 0.7189 - val_loss: 0.5623 - val_acc: 0.7142
Epoch 91/1500
 - 14s - loss: 0.5346 - acc: 0.7185 - val_loss: 0.5626 - val_acc: 0.7158
Epoch 92/1500
 - 14s - loss: 0.5342 - acc: 0.7190 - val_loss: 0.5620 - val_acc: 0.7146
Epoch 93/1500
 - 14s - loss: 0.5336 - acc: 0.7193 - val_loss: 0.5614 - val_acc: 0.7147
Epoch 94/1500
 - 14s - loss: 0.5327 - acc: 0.7205 - val_loss: 0.5622 - val_acc: 0.7153
Epoch 95/1500
 - 14s - loss: 0.5327 - acc: 

Epoch 178/1500
 - 14s - loss: 0.5027 - acc: 0.7379 - val_loss: 0.5419 - val_acc: 0.7239
Epoch 179/1500
 - 14s - loss: 0.5024 - acc: 0.7386 - val_loss: 0.5431 - val_acc: 0.7247
Epoch 180/1500
 - 14s - loss: 0.5022 - acc: 0.7385 - val_loss: 0.5436 - val_acc: 0.7243
Epoch 181/1500
 - 14s - loss: 0.5020 - acc: 0.7386 - val_loss: 0.5429 - val_acc: 0.7246
Epoch 182/1500
 - 14s - loss: 0.5014 - acc: 0.7391 - val_loss: 0.5431 - val_acc: 0.7251
Epoch 183/1500
 - 14s - loss: 0.5020 - acc: 0.7385 - val_loss: 0.5430 - val_acc: 0.7250
Epoch 184/1500
 - 14s - loss: 0.5011 - acc: 0.7390 - val_loss: 0.5400 - val_acc: 0.7262
Epoch 185/1500
 - 14s - loss: 0.5006 - acc: 0.7389 - val_loss: 0.5414 - val_acc: 0.7265
Epoch 186/1500
 - 14s - loss: 0.5007 - acc: 0.7393 - val_loss: 0.5415 - val_acc: 0.7256
Epoch 187/1500
 - 14s - loss: 0.5015 - acc: 0.7389 - val_loss: 0.5413 - val_acc: 0.7262
Epoch 188/1500
 - 14s - loss: 0.4997 - acc: 0.7405 - val_loss: 0.5417 - val_acc: 0.7267
Epoch 189/1500
 - 14s - loss: 0.

 - 14s - loss: 0.4792 - acc: 0.7529 - val_loss: 0.5315 - val_acc: 0.7307
Epoch 272/1500
 - 14s - loss: 0.4790 - acc: 0.7530 - val_loss: 0.5308 - val_acc: 0.7310
Epoch 273/1500
 - 14s - loss: 0.4786 - acc: 0.7542 - val_loss: 0.5300 - val_acc: 0.7303
Epoch 274/1500
 - 14s - loss: 0.4788 - acc: 0.7535 - val_loss: 0.5309 - val_acc: 0.7311
Epoch 275/1500
 - 14s - loss: 0.4788 - acc: 0.7533 - val_loss: 0.5305 - val_acc: 0.7317
Epoch 276/1500
 - 14s - loss: 0.4785 - acc: 0.7536 - val_loss: 0.5306 - val_acc: 0.7315
Epoch 277/1500
 - 14s - loss: 0.4775 - acc: 0.7538 - val_loss: 0.5309 - val_acc: 0.7315
Epoch 278/1500
 - 14s - loss: 0.4777 - acc: 0.7547 - val_loss: 0.5309 - val_acc: 0.7324
Epoch 279/1500
 - 14s - loss: 0.4769 - acc: 0.7546 - val_loss: 0.5300 - val_acc: 0.7315
Epoch 280/1500
 - 14s - loss: 0.4771 - acc: 0.7553 - val_loss: 0.5312 - val_acc: 0.7307
Epoch 281/1500
 - 14s - loss: 0.4767 - acc: 0.7550 - val_loss: 0.5306 - val_acc: 0.7310
Epoch 282/1500
 - 14s - loss: 0.4767 - acc: 0.7

Epoch 365/1500
 - 14s - loss: 0.4615 - acc: 0.7653 - val_loss: 0.5243 - val_acc: 0.7350
Epoch 366/1500
 - 14s - loss: 0.4608 - acc: 0.7660 - val_loss: 0.5244 - val_acc: 0.7353
Epoch 367/1500
 - 14s - loss: 0.4601 - acc: 0.7659 - val_loss: 0.5253 - val_acc: 0.7343
Epoch 368/1500
 - 14s - loss: 0.4610 - acc: 0.7651 - val_loss: 0.5248 - val_acc: 0.7341
Epoch 369/1500
 - 14s - loss: 0.4604 - acc: 0.7656 - val_loss: 0.5243 - val_acc: 0.7349
Epoch 370/1500
 - 14s - loss: 0.4594 - acc: 0.7664 - val_loss: 0.5244 - val_acc: 0.7349
Epoch 371/1500
 - 14s - loss: 0.4593 - acc: 0.7668 - val_loss: 0.5249 - val_acc: 0.7339
Epoch 372/1500
 - 14s - loss: 0.4596 - acc: 0.7665 - val_loss: 0.5241 - val_acc: 0.7344
Epoch 373/1500
 - 14s - loss: 0.4586 - acc: 0.7666 - val_loss: 0.5238 - val_acc: 0.7356
Epoch 374/1500
 - 14s - loss: 0.4586 - acc: 0.7669 - val_loss: 0.5240 - val_acc: 0.7347
Epoch 375/1500
 - 14s - loss: 0.4586 - acc: 0.7662 - val_loss: 0.5237 - val_acc: 0.7344
Epoch 376/1500
 - 14s - loss: 0.

...

 - 11s - loss: 0.3919 - acc: 0.8093 - val_loss: 0.5199 - val_acc: 0.7389
Epoch 1112/1500
 - 11s - loss: 0.3923 - acc: 0.8095 - val_loss: 0.5195 - val_acc: 0.7388
Epoch 1113/1500
 - 11s - loss: 0.3920 - acc: 0.8096 - val_loss: 0.5197 - val_acc: 0.7380
Epoch 1114/1500
 - 11s - loss: 0.3919 - acc: 0.8097 - val_loss: 0.5199 - val_acc: 0.7374
Epoch 1115/1500
 - 11s - loss: 0.3924 - acc: 0.8098 - val_loss: 0.5199 - val_acc: 0.7386
Epoch 1116/1500
 - 11s - loss: 0.3914 - acc: 0.8100 - val_loss: 0.5194 - val_acc: 0.7394
Epoch 1117/1500
 - 11s - loss: 0.3925 - acc: 0.8096 - val_loss: 0.5199 - val_acc: 0.7381
Epoch 1118/1500
 - 11s - loss: 0.3928 - acc: 0.8092 - val_loss: 0.5201 - val_acc: 0.7382
Epoch 1119/1500
 - 11s - loss: 0.3914 - acc: 0.8103 - val_loss: 0.5197 - val_acc: 0.7377
Epoch 1120/1500
 - 11s - loss: 0.3921 - acc: 0.8100 - val_loss: 0.5195 - val_acc: 0.7382
Epoch 1121/1500
 - 11s - loss: 0.3924 - acc: 0.8091 - val_loss: 0.5199 - val_acc: 0.7389
Epoch 1122/1500
 - 11s - loss: 0.3917

Epoch 1204/1500
 - 11s - loss: 0.3878 - acc: 0.8123 - val_loss: 0.5213 - val_acc: 0.7375
Epoch 1205/1500
 - 11s - loss: 0.3886 - acc: 0.8119 - val_loss: 0.5210 - val_acc: 0.7380
Epoch 1206/1500
 - 11s - loss: 0.3884 - acc: 0.8121 - val_loss: 0.5206 - val_acc: 0.7377
Epoch 1207/1500
 - 11s - loss: 0.3879 - acc: 0.8123 - val_loss: 0.5208 - val_acc: 0.7380
Epoch 1208/1500
 - 11s - loss: 0.3880 - acc: 0.8127 - val_loss: 0.5208 - val_acc: 0.7378
Epoch 1209/1500
 - 11s - loss: 0.3875 - acc: 0.8125 - val_loss: 0.5212 - val_acc: 0.7374
Epoch 1210/1500
 - 11s - loss: 0.3885 - acc: 0.8115 - val_loss: 0.5212 - val_acc: 0.7369
Epoch 1211/1500
 - 11s - loss: 0.3880 - acc: 0.8124 - val_loss: 0.5207 - val_acc: 0.7364
Epoch 1212/1500
 - 11s - loss: 0.3867 - acc: 0.8130 - val_loss: 0.5208 - val_acc: 0.7384
Epoch 1213/1500
 - 11s - loss: 0.3873 - acc: 0.8127 - val_loss: 0.5216 - val_acc: 0.7372
Epoch 1214/1500
 - 11s - loss: 0.3871 - acc: 0.8128 - val_loss: 0.5205 - val_acc: 0.7378
Epoch 1215/1500
 - 11

 - 11s - loss: 0.3849 - acc: 0.8143 - val_loss: 0.5208 - val_acc: 0.7375
Epoch 1297/1500
 - 11s - loss: 0.3848 - acc: 0.8140 - val_loss: 0.5214 - val_acc: 0.7386
Epoch 1298/1500
 - 11s - loss: 0.3833 - acc: 0.8150 - val_loss: 0.5219 - val_acc: 0.7374
Epoch 1299/1500
 - 11s - loss: 0.3840 - acc: 0.8149 - val_loss: 0.5217 - val_acc: 0.7382
Epoch 1300/1500
 - 11s - loss: 0.3841 - acc: 0.8144 - val_loss: 0.5214 - val_acc: 0.7389
Epoch 1301/1500
 - 11s - loss: 0.3833 - acc: 0.8148 - val_loss: 0.5213 - val_acc: 0.7388
Epoch 1302/1500
 - 11s - loss: 0.3836 - acc: 0.8150 - val_loss: 0.5212 - val_acc: 0.7376
Epoch 1303/1500
 - 11s - loss: 0.3827 - acc: 0.8157 - val_loss: 0.5207 - val_acc: 0.7379
Epoch 1304/1500
 - 11s - loss: 0.3836 - acc: 0.8148 - val_loss: 0.5211 - val_acc: 0.7390
Epoch 1305/1500
 - 11s - loss: 0.3838 - acc: 0.8152 - val_loss: 0.5211 - val_acc: 0.7378
Epoch 1306/1500
 - 11s - loss: 0.3837 - acc: 0.8153 - val_loss: 0.5222 - val_acc: 0.7377
Epoch 1307/1500
 - 11s - loss: 0.3838

Epoch 1389/1500
 - 11s - loss: 0.3798 - acc: 0.8172 - val_loss: 0.5214 - val_acc: 0.7373
Epoch 1390/1500
 - 11s - loss: 0.3797 - acc: 0.8173 - val_loss: 0.5219 - val_acc: 0.7376
Epoch 1391/1500
 - 11s - loss: 0.3796 - acc: 0.8171 - val_loss: 0.5222 - val_acc: 0.7367
Epoch 1392/1500
 - 11s - loss: 0.3802 - acc: 0.8167 - val_loss: 0.5211 - val_acc: 0.7394
Epoch 1393/1500
 - 11s - loss: 0.3792 - acc: 0.8171 - val_loss: 0.5226 - val_acc: 0.7383
Epoch 1394/1500
 - 11s - loss: 0.3797 - acc: 0.8178 - val_loss: 0.5223 - val_acc: 0.7385
Epoch 1395/1500
 - 11s - loss: 0.3792 - acc: 0.8172 - val_loss: 0.5226 - val_acc: 0.7372
Epoch 1396/1500
 - 11s - loss: 0.3804 - acc: 0.8166 - val_loss: 0.5226 - val_acc: 0.7380
Epoch 1397/1500
 - 11s - loss: 0.3801 - acc: 0.8167 - val_loss: 0.5222 - val_acc: 0.7382
Epoch 1398/1500
 - 11s - loss: 0.3798 - acc: 0.8173 - val_loss: 0.5221 - val_acc: 0.7375
Epoch 1399/1500
 - 11s - loss: 0.3804 - acc: 0.8170 - val_loss: 0.5213 - val_acc: 0.7379
Epoch 1400/1500
 - 11

Epoch 1482/1500
 - 8s - loss: 0.3776 - acc: 0.8187 - val_loss: 0.5217 - val_acc: 0.7370
Epoch 1483/1500
 - 8s - loss: 0.3763 - acc: 0.8183 - val_loss: 0.5233 - val_acc: 0.7365
Epoch 1484/1500
 - 8s - loss: 0.3750 - acc: 0.8200 - val_loss: 0.5228 - val_acc: 0.7371
Epoch 1485/1500
 - 8s - loss: 0.3764 - acc: 0.8194 - val_loss: 0.5230 - val_acc: 0.7361
Epoch 1486/1500
 - 8s - loss: 0.3759 - acc: 0.8198 - val_loss: 0.5229 - val_acc: 0.7376
Epoch 1487/1500
 - 8s - loss: 0.3761 - acc: 0.8196 - val_loss: 0.5233 - val_acc: 0.7374
Epoch 1488/1500
 - 8s - loss: 0.3755 - acc: 0.8195 - val_loss: 0.5231 - val_acc: 0.7350
Epoch 1489/1500
 - 8s - loss: 0.3761 - acc: 0.8193 - val_loss: 0.5224 - val_acc: 0.7369
Epoch 1490/1500
 - 8s - loss: 0.3756 - acc: 0.8189 - val_loss: 0.5223 - val_acc: 0.7381
Epoch 1491/1500
 - 8s - loss: 0.3756 - acc: 0.8194 - val_loss: 0.5233 - val_acc: 0.7362
Epoch 1492/1500
 - 8s - loss: 0.3762 - acc: 0.8194 - val_loss: 0.5221 - val_acc: 0.7369
Epoch 1493/1500
 - 8s - loss: 0.